# Acculturation gap for interface language
Looking at how different the language of comments are vs. the language of interfaces (posts + guidelines)

In [38]:
import sys
sys.path.append('../lib/')

# do it this way so you have all the functions without SLM_building.[....]
%run -n ../../lib/SLM_building.py

directory already ../data/test_subs_2017/


In [19]:
df_top_posts = pd.read_csv('../cleaned/top_posts_cleaned.csv')
df_sub = pd.read_csv('../cleaned/subbreddits.csv').fillna(' ')

In [20]:
# Train monthly set of SLMs from subreddit comments 

# Take the final month (or all of them) and compare them 50 randomly drawn 10 word spans from the full_descr + submit text

# Do the same for the top post titles (50 drawn randomly)


In [21]:
##############################################
# Functions for cleaning the interface text 
# of subreddits
# making sure this is procossed EXACTLY the same way
# as in SLM_building
##############################################

# chunks list into list of lists with span_len,
# cuts anything off that isn't exactly span_len
def chunk(s, span_len):
    span_len = max(1, span_len)
    chunked_list = (s[i:i+span_len] for i in range(0, len(s), span_len))
    return [c for c in chunked_list if len(c) == span_len]



# collect 25 randomly chose 10 word spans from full description (+ submit_text) of subreddit
# assumes sub_text is tokenized
def get_sub_interface_text(sub_text, num_posts=25, span_len=10):
    chunked_sub_text = chunk(sub_text, span_len)
    sampled_text = random.sample(chunked_sub_text, 25)
    return sampled_text


# collect randomly chose 10 word spans from sub top posts
def get_sub_top_posts_text(df_sub, text_col='title', num_posts=25, span_len=10):
    sampled_text = df_sub[text_col].sample(num_posts)
    # random span tokenizes
    sampled_spans = sampled_text.apply(lambda x: get_random_span(x, span_len))
    return list(sampled_spans.values)
    

In [22]:
groups = df_top_posts.groupby('subreddit_name')
sub_text = {}
for sub, g in groups:
    sub_text[sub] = get_sub_top_posts_text(g, num_posts=40)

In [23]:
sub_text

{'BabyBumps': [[')',
   'is',
   'trying',
   'to',
   'name',
   'my',
   '(',
   'f31',
   ')',
   'new'],
  ['to',
   'make',
   'the',
   'journey',
   'to',
   'our',
   'arms',
   '.',
   'hopefully',
   'after'],
  ['had', 'a', 'child', '?', 'welcome', 'to', 'hell', '!', '!', '!'],
  ['been',
   'asking',
   'myself',
   'this',
   'question',
   'lately',
   'as',
   'my',
   'appetite',
   'is'],
  ['could',
   'be',
   'described',
   'as',
   '``',
   'unchaperoned',
   'child',
   'at',
   'a',
   'birthday'],
  ['me', 'up', 'with', 'flowers', ',', 'a', 'maroon', 'cardigan', 'that', '’'],
  ['been',
   'making',
   'illustrations',
   'inspired',
   'by',
   'the',
   'adventure',
   'that',
   'is',
   'being'],
  ['when',
   'your',
   'dog',
   'is',
   'walking',
   'around',
   'the',
   'bed',
   'and',
   'steps'],
  ['that',
   'baby',
   'girls',
   'clothing',
   'is',
   'so',
   'much',
   'cuter',
   'than',
   'baby'],
  ['for', 'me', '&', 'i', "'m", 'still', 

In [37]:
subs = ['Naruto', 'pics', 'BabyBumps', 'science', 'politics', 'Cooking']
slms_subs = {}
# build SLMs only for final month
for i, s in enumerate(subs):
    df_comments, df_author_counts = import_csvs(s)
    slms_subs[s] = build_SLMs(df_comments, df_author_counts, 10, month=12)
    

Importing  ../cleaned/sub_comments/Naruto_comments_2017.csv ..... Done
Importing  ../cleaned/sub_author_counts/Naruto_author_counts_2017.csv ..... Done
Creating  10 SLMs  for  month 12 ..... Done
Importing  ../cleaned/sub_comments/pics_comments_2017.csv ..... Done
Importing  ../cleaned/sub_author_counts/pics_author_counts_2017.csv ..... Done
Creating  10 SLMs  for  month 12 ..... Done
Importing  ../cleaned/sub_comments/BabyBumps_comments_2017.csv ..... Done
Importing  ../cleaned/sub_author_counts/BabyBumps_author_counts_2017.csv ..... Done
Creating  10 SLMs  for  month 12 ..... Done
Importing  ../cleaned/sub_comments/science_comments_2017.csv ..... Done
Importing  ../cleaned/sub_author_counts/science_author_counts_2017.csv ..... Done
Creating  10 SLMs  for  month 12 ..... Done
Importing  ../cleaned/sub_comments/politics_comments_2017.csv ..... Done
Importing  ../cleaned/sub_author_counts/politics_author_counts_2017.csv ..... Done
Creating  10 SLMs  for  month 12 ..... Done
Importing  .

In [40]:
slms_subs

{'Naruto': [<nltk.lm.models.Laplace at 0x7f67017f46d8>,
 'pics': [<nltk.lm.models.Laplace at 0x7f66f922e898>,
 'BabyBumps': [<nltk.lm.models.Laplace at 0x7f668b5f75c0>,
 'science': [<nltk.lm.models.Laplace at 0x7f6685a6d9e8>,
 'politics': [<nltk.lm.models.Laplace at 0x7f66f1b473c8>,
 'Cooking': [<nltk.lm.models.Laplace at 0x7f65709b0160>,
  <nltk.lm.models.Laplace at 0x7f6606e56d30>]}

In [41]:
gaps = {}
for k in slms_subs.keys():
    gaps[k] = {}
    for j in slms_subs.keys():
        # using keys again because it is a subset but use k as way if interating on sub text
        gaps[k][j] = np.mean(calc_single_acc_gap(slms_subs[j], sub_text[k]))
        

In [42]:
pd.DataFrame(gaps)

{'Naruto': {'Naruto': 10.33575970226463,
  'pics': 10.58090004253057,
  'BabyBumps': 10.316574425916597,
  'science': 10.63742163359529,
  'politics': 10.614648196157278,
  'Cooking': 10.541290810680447},
 'pics': {'Naruto': 10.391095833474369,
  'pics': 10.552009698588327,
  'BabyBumps': 10.351498189259665,
  'science': 10.565179368378196,
  'politics': 10.55270113087372,
  'Cooking': 10.53265412394289},
 'BabyBumps': {'Naruto': 10.455245085552493,
  'pics': 10.632009383650836,
  'BabyBumps': 10.346914203718061,
  'science': 10.659807452003939,
  'politics': 10.66288734586009,
  'Cooking': 10.600526002111247},
 'science': {'Naruto': 10.630824704262968,
  'pics': 10.802086986146819,
  'BabyBumps': 10.63175644454745,
  'science': 10.767936412310041,
  'politics': 10.811099292695927,
  'Cooking': 10.785751408995061},
 'politics': {'Naruto': 10.724094589763112,
  'pics': 10.883509494853872,
  'BabyBumps': 10.72154986980764,
  'science': 10.91035709080414,
  'politics': 10.831802330882756,

In [26]:
gaps

{'Naruto': {'Naruto': 10.309566253957087,
  'pics': 10.599818879297757,
  'BabyBumps': 10.321813156607478,
  'science': 10.635057059555006,
  'politics': 10.634222223622967,
  'Cooking': 10.48257035071183},
 'pics': {'Naruto': 10.375989206933006,
  'pics': 10.594172726910823,
  'BabyBumps': 10.378810683296047,
  'science': 10.538452019693443,
  'politics': 10.5659308801206,
  'Cooking': 10.515605172623196},
 'BabyBumps': {'Naruto': 10.469333093878367,
  'pics': 10.628594135457359,
  'BabyBumps': 10.34359358283957,
  'science': 10.663440114909443,
  'politics': 10.646855212435955,
  'Cooking': 10.54461176848387},
 'science': {'Naruto': 10.62909429540677,
  'pics': 10.82440914596649,
  'BabyBumps': 10.689996348573615,
  'science': 10.769215878572453,
  'politics': 10.806952661473447,
  'Cooking': 10.754795074038254},
 'politics': {'Naruto': 10.713879632157653,
  'pics': 10.952211544219107,
  'BabyBumps': 10.758810290830771,
  'science': 10.925067847716281,
  'politics': 10.83005901122066

In [116]:
for k in sub_text.keys():
    print(k)

BabyBumps
Cooking
Economics
Naruto
depression
pics
politics
science


In [107]:
sub_text

{'BabyBumps': [['after',
   'few',
   'weeks',
   'in',
   'NICU',
   'we',
   'can',
   'bring',
   'baby',
   'girl'],
  ['Introduction',
   'and',
   'Daily',
   'Bump',
   'Picture',
   'Thread',
   '-',
   'February',
   '08',
   ','],
  ['I', 'need', 'to', 'vent', '.', 'A', 'cousin', 'I', '’', 'm'],
  ['I', 'humble', 'brag', 'because', 'my', 'bf', 'woke', 'me', 'up', 'with'],
  ['to',
   'have',
   'received',
   'my',
   'free',
   'registry',
   'package',
   'from',
   'amazon',
   '.'],
  ['dog',
   'is',
   'walking',
   'around',
   'the',
   'bed',
   'and',
   'steps',
   'on',
   'your'],
  ['and',
   'Daily',
   'Bump',
   'Picture',
   'Thread',
   '-',
   'February',
   '09',
   ',',
   '2019'],
  ['(', 'I', "'m", 'currently', '19w+3', ')', '.', 'The', 'ladies', 'in'],
  ['is', '...', '”', 'I', '’', 'm', 'going', 'to', 'lose', 'my'],
  ['10:24am', 'this', 'morning', 'by', 'c', 'section', '.', '5', 'lbs', '4oz'],
  ['Update', 'on', 'me', ',', 'the', 'one', 'with', 'Sta

In [85]:
sub_text

{'BabyBumps': 458    [My, mother, is, not, supportive, and, tries, ...
 332    [rant, ], I, want, to, kick, my, husband, in, ...
 365    [Introduction, and, Daily, Bump, Picture, Thre...
 48     [and, Daily, Bump, Picture, Thread, -, Februar...
 97     [Weekly, Ultrasound, and, Announcement, Thread...
 550    [pregnancy, is, hard, and, you, cant, sleep, n...
 57     [peacefully, for, the, first, time, in, a, few...
 477    [Alice, was, born, on, February, 5th, at, 39, ...
 499    [Weekly, Ultrasound, and, Announcement, Thread...
 219    [today, could, be, described, as, ``, unchaper...
 41     [your, dog, is, walking, around, the, bed, and...
 285    [me, this, ., Probably, an, oldie, but, I, had...
 396    [32, weeks, pregnant, and, fell, on, ice, and,...
 96     [Introduction, and, Daily, Bump, Picture, Thre...
 341    [Having, my, little, girl, in, 4, days, ., Her...
 150    [Introduction, and, Daily, Bump, Picture, Thre...
 119    [Final, Update, ], We, haven, ’, t, bought, a,...
 

In [35]:
len(get_sub_interface_text(word_tokenize(df_sub.full_descr.iloc[0])))

['#', '[', 'Submission', 'Rules', ']', '(', 'https', ':', '//www.reddit.com/r/science/wiki/rules', '#', 'wiki_submission_rules', ')', '1', '.', 'Directly', 'link', 'to', 'published', 'peer-reviewed', 'research', 'or', 'media', 'summary', '2', '.', 'No', 'summaries', 'of', 'summaries', ',', 're-hosted', 'press', 'releases', ',', 'reviews', ',', 'or', 'reposts', '3', '.', 'Research', 'must', 'be', 'less', 'than', '6', 'months', 'old', '4', '.', 'No', 'editorialized', ',', 'sensationalized', ',', 'or', 'biased', 'titles', '5', '.', 'No', 'blogspam', ',', 'images', ',', 'videos', ',', 'or', 'infographics', '6', '.', 'All', 'submissions', 'must', 'have', 'flair', 'assigned', '#', '[', 'Comment', 'Rules', ']', '(', 'https', ':', '//www.reddit.com/r/science/wiki/rules', '#', 'wiki_comment_rules', ')', '1', '.', 'No', 'off-topic', 'comments', ',', 'memes', ',', 'or', 'jokes', '2', '.', 'No', 'abusive', ',', 'offensive', ',', 'or', 'spam', 'comments', '3', '.', 'Non-professional', 'personal', '

25

In [24]:
df_sub[['descr', 'public_descr', 'submit_text', 'full_descr']] = df_sub[['descr', 'public_descr', 'submit_text', 'full_descr']].apply(lambda x: x.apply(word_tokenize), axis=1)

,descr,public_descr,submit_text,full_descr
0,"[#, [, Submission, Rules, ], (, https, :, //ww...","[This, community, is, a, place, to, share, and...",[],"[#, [, Submission, Rules, ], (, https, :, //ww..."
1,"[#, #, **Welcome, to, /r/Politics, !, Please, ...","[/r/Politics, is, for, news, and, discussion, ...","[1, ., Submissions, must, be, for, **Current, ...","[#, #, **Welcome, to, /r/Politics, !, Please, ..."
2,"[#, #, #, Subreddit, Rules, --, I, ., **Discip...","[News, and, discussion, about, economics, ,, f...","[**Self-post, privileges, require, ``, Bureau,...","[#, #, #, Subreddit, Rules, --, I, ., **Discip..."
3,"[#, #, A, supportive, space, for, anyone, stru...","[Peer, support, for, anyone, struggling, with,...","[If, you, 're, having, thoughts, of, suicide, ...","[#, #, A, supportive, space, for, anyone, stru..."
4,"[#, #, #, #, #, Please, read, these, 1, ., All...","[/r/Cooking, is, a, place, for, the, cooks, of...","[**1, ., If, you, are, asking, for, help, be, ...","[#, #, #, #, #, Please, read, these, 1, ., All..."
5,"[A, place, to, share, photographs, and, pictur...","[A, place, to, share, photographs, and, pictur...","[Please, read, [, the, sidebar, ], (, /r/pics/...","[A, place, to, share, photographs, and, pictur..."
6,"[1, ., [, Boruto, Chapter, 31, Free, (, VIZ, )...","[Everything, related, to, Naruto, and, Boruto,...","[**WE, 'VE, UPDATED, OUR, RULES, !, [, READ, T...","[1, ., [, Boruto, Chapter, 31, Free, (, VIZ, )..."
7,"[#, #, #, All, Bump, Photos, belong, in, our, ...","[A, place, for, pregnant, redditors, ,, those,...","[Our, most, commonly, broken, rules, :, *, **D...","[#, #, #, All, Bump, Photos, belong, in, our, ..."
